# 图像特效：

- 特效1：灰度处理
- 特效2：底板效果
- 特效3：马赛克
- 特效4：毛玻璃效果
- 特效5：图像融合
- 特效6：图片蓝色
- 特效7：边缘检测
- 特效8：浮雕效果


- 学习步骤：
    1. API调用
    2. 图像特效实现的基本算法原理
    3. 源码的形式，根据算法原理实现图像特效

In [17]:
import cv2
import numpy as np
import math

img = cv2.imread("./eunha.jpg",1)

height = img.shape[0]
width = img.shape[1]
channel = img.shape[2]

# 1. 图片的灰度处理

- 灰度，最重要

- 图像检测，以灰度图像作为基础
    - 图像的边缘检测
    - 人脸检测
    - 行人识别

- 算法的实时性
    - 计算机视觉都是实时处理，所以【算法优化】很重要

In [2]:
# 方法一：imread
gray_imread = cv2.imread("./eunha.jpg",0)
cv2.imshow("1. gray_imread", gray_imread)



# 方法二：颜色空间转换
gray_cvt = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)    # 参数1：原图数据data，参数2：转换方式bgr-gray
cv2.imshow("2. gray_cvt", gray_cvt)



# 方法三：通过算法原理，使用源码实现灰度处理
# r=g=b=gray; gray = (b+g+r)/3 避免溢出

gray_code = np.zeros(img.shape, np.uint8)    # 新建模板

for i in range(0, height):
    for j in range(0, width):
        b,g,r = img[i,j]    # 读取每个像素点的像素值
        # 先转换成int类型，防止计算时出现越界
        gray = (int(b)+int(g)+int(r)) / 3
        gray_code[i,j] = np.uint8(gray)
        # dst[i,j] = (int(b)+int(g)+int(r)) / 3
cv2.imshow("3. gray_coding", gray_code)



# 方法四：心理学公式
# Gray = R*0.299 + G*0.587 + B*0.114

gray_psy = np.zeros(img.shape, np.uint8)    # 新建模板

for i in range(0, height):
    for j in range(0, width):
        b,g,r = img[i,j]
        # 先转换成int类型，防止计算时出现越界
        b = int(b)
        g = int(g)
        r = int(r)
        # gray = b*0.114 + g*0.587 + g*0.299
        # 优化算法，定点运算 > 浮点运算
        gray = (b*1 + g*2 + g*1)/4
        gray_psy[i,j] = np.uint8(gray)

cv2.imshow("4. gray_psy", gray_psy)



cv2.imshow("Original", img)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# 2. 算法优化

- 定点运算 > 浮点运算
- +/- > x/÷
    - 乘以的数越大，误差越小
- 移位操作 > x/÷
    - 放大2倍(x2)：左移1位(<<1)
    - 缩小4倍(÷4)：右移2位(>>2)
- 实现图片灰度图像的优化
    - 原始：
        > gray = b*0.114 + g*0.587 + g*0.299
    - 优化算法：浮点运算 -> 定点运算 
        > gray = (b*1 + g*2 + g*1)/4
    - 优化算法：定点运算 -> 移位运算
        > gray = (b + (g<<1) + g*1)>>2       

In [3]:
for i in range(0, height):
    for j in range(0, width):
        b,g,r = img[i,j]
        # 先转换成int类型，防止计算时出现越界
        b = int(b)
        g = int(g)
        r = int(r)
        # gray = b*0.114 + g*0.587 + g*0.299
        # gray = (b*1 + g*2 + g*1)/4      # 优化算法：浮点运算 -> 定点运算  
        gray = (b + (g<<1) + g*1)>>2      # 优化算法：定点运算 -> 移位运算
        gray_psy[i,j] = np.uint8(gray)

cv2.imshow("optimization_psy", gray_psy)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# 3. 图片颜色反转

- 灰色图片的颜色反转：
    - 反转灰度值 = 255 - 当前灰度值

- 彩色图片的颜色反转：
    - 反转b/g/r值 = 255 - 当前b/g/r值

In [4]:
# 灰色图片的颜色反转：
# 反转灰度值 = 255 - 当前灰度值

gray_reverse = np.zeros((height,width,1), np.uint8)

for i in range(0, height):
    for j in range(0, width):
        gray_scale = gray_cvt[i,j]
        gray_reverse[i,j] = 255 - gray_scale


# 彩色图片的颜色反转：
# 反转b/g/r值 = 255 - 当前b/g/r值

img_reverse = np.zeros(img.shape, np.uint8)

for i in range(0, height):
    for j in range(0, width):
        b,g,r = img[i,j]
        img_reverse[i,j] = (255-b, 255-g, 255-r)
        
        

cv2.imshow("gray_reverse", gray_reverse)
cv2.imshow("img_reverse", img_reverse)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# 4. 马赛克效果

- 用10x10矩形方块中的左上角的一个像素点，替换掉10x10中的100个像素点，让这100个像素保持一致

In [5]:
# 遍历马赛克效果范围内的所有像素点
img_mosaic = img.copy()
for m in range(385,503):
    for n in range(222,345):
        if m%10==0 and n%10==0:    # 每隔10个宽高的矩形，取一个像素点
            b,g,r = img[m,n]
            for i in range(0,10):
                for j in range(0,10):
                    img_mosaic[i+m,j+n] = (b,g,r)
                    
cv2.imshow("img_mosaic", img_mosaic)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# 5. 毛玻璃效果

- 在特定矩阵中【随机】选取一个像素点，替换掉特定矩阵中所有的像素点

In [6]:
import random
dst = np.zeros((img.shape), np.uint8)

for i in range(0, height-8):    # 有可能超出图片的范围，所以要减去随机数的最大值
    for j in range(0, width-8):
        random_int = random.randint(0,8)     # 0-8之间的随机数
        (b,g,r) = img[i + random_int, j + random_int]
        dst[i,j] = (b,g,r)
        
        
# 图片最右边和最下边会有黑边，是因为-8后，没有操作图片，0填充
cv2.imshow("woolen_glass", dst)
cv2.waitKey(5000)
cv2.destroyAllWindows()        

# 6. 图片融合

- 每张图片，乘以比例系数，然后相加
    - dst = img1 * alpha + img2 * (1-alpha)
        - alpha系数，0.3
        
- 融合的图片，大小一致


In [34]:
img1 = cv2.imread("./sinb.jpg",1)

dst = np.zeros(img.shape, np.uint8)
# 权重相加：
dst = cv2.addWeighted(img,0.5,img1,0.5,0)
# 参数1：原图数据1
# 参数2：alpha参数
# 参数1：原图数据2
# 参数2：1 - alpha参数


# ROI
roiH = int(height/2)
roiW = int(width/2)
img0ROI = img[385:503, 222:345]
img1ROI = img1[385:503, 222:345]
dst_roi = np.zeros((roiH, roiW, 3), np.uint8)
dst_roi = cv2.addWeighted(img0ROI,0.5,img1ROI,0.5,0)


cv2.imshow("merge_all", dst)
cv2.imshow("merge_roi", dst_roi)


cv2.waitKey(5000)
cv2.destroyAllWindows() 

# 7. 边缘检测

- 原理：图像的卷积运算
- 步骤：
    - 把彩色图转成灰度图
    - 高斯滤波
    - canny方法，完成边缘检测

In [8]:
# 1. 转换成灰度图：gray_cvt
# 2. 高斯滤波
imgG = cv2.GaussianBlur(gray_cvt,(3,3),0)

# 3. 调用canny方法
dst = cv2.Canny(img,50,50)
# 参数1：图片的数据
# 参数2：门线，图片经过卷积之后的值，如果大于门线值，那么就认为是边缘检测

cv2.imshow("canny", dst)

cv2.waitKey(5000)
cv2.destroyAllWindows()

# 7.1 sobel算法原理：

- sobel算子
    - 也叫 Sobel 滤波, 是两个 3*3 的矩阵, 主要用来计算图像中某一点在横向/纵向上的梯度
    - 相当于2个携带固定权重的卷积核
    - 水平矩阵：[[-1,0,1],[-2,0,2],[-1,0,1]]
    - 竖直矩阵：[[-1,-2,-1],[0,0,0],[1,2,1]]
    
- 使用sobel算子与图片进行卷积运算，得到一个值。若此值大于门线值，则是边缘；小于门线值则是非边缘

- 计算：
    - 水平方向：gx = [[-1,0,1],[-2,0,2],[-1,0,1]] * gray_pic
    - 竖直方向：gx = [[-1,-2,-1],[0,0,0],[1,2,1]] * gray_pic


In [24]:
dst_sobal = np.zeros((height,width,1),np.uint8)

for i in range(0, height - 2):    # 3x3 sobel算子，防止卷积核溢出。由于最大下标是[2]，所以高宽-2
    for j in range(0, width - 2): 
        gy = gray_cvt[i,j]*(-1) + gray_cvt[i,j+1]*(-2) + gray_cvt[i,j+2]*(-1) + gray_cvt[i+2,j]*1 + gray_cvt[i+2,j+1]*2 + gray_cvt[i+2,j+2]*1    # 竖直方向的梯度
        gx = gray_cvt[i,j]*(-1) + gray_cvt[i,j+2]*1 + gray_cvt[i+1,j]*(-2) + gray_cvt[i+1,j+2]*2 + gray_cvt[i+2,j]*(-1) + gray_cvt[i+2,j+2]*1    # 水平方向的梯度
        
        grad = math.sqrt(gx*gx + gy*gy)    # 梯度
            
        if grad > 50:    # 50为门线值，大于50则是边缘，小于50则是非边缘
            dst_sobal[i,j] = 255
        else:
            dst_sobal[i,j] = 0

cv2.imshow("sobel", dst_sobal)
cv2.waitKey(5000)
cv2.destroyAllWindows()            

# 8. 浮雕效果

- 原理：
    - newP = gray0 - gray1 + 150
        - 相邻像素的相减，是为了突出灰度的突变，即边缘特征
        - +150是为了增强图片浮雕效果的灰度等级


In [30]:
dst_cameo = np.zeros((height,width,1),np.uint8)


for i in range(0,height):
    for j in range(0,width-1):    # 因为纵坐标+1，所以循环遍历的宽度要-1。否则数组(矩阵)越界
        current_pixel = int(gray_cvt[i,j])    # 当前像素，数据处理过程中，可能超过0-255，所以先强制转换
        next_pixel = int(gray_cvt[i,j+1])    # 下一个像素
        new_pixel = current_pixel - next_pixel + 150
        new_pixel = np.where(new_pixel>255, 255, new_pixel)
        new_pixel = np.where(new_pixel<0, 0, new_pixel)
        dst_cameo[i,j] = new_pixel
        
        
cv2.imshow("cameo", dst_cameo)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# 9. 颜色风格

- 颜色的映射变换，实现不同的风格效果

    1. 颜色映射公式
        - 方便使用
        - 不方便拟合颜色
    2. 创建rgb查找表：
        - 用新的rgb代替旧的rgb

In [33]:
# 让图片的蓝色更蓝：b=b*1.5，g=g*1.3

dst_colorStyle = np.zeros_like(img)

for i in range(0, height):
    for j in range(0, width):
        (b,g,r) = img[i,j]
        b = b * 1.5
#         g = g * 1.3
        b = np.where(b>255,255,0)
#         g = np.where(g>255,255,0)
        dst_colorStyle[i,j] = (b,g,r)
        
cv2.imshow("original", img)
cv2.imshow("enhancBlue", dst_colorStyle)
cv2.waitKey(5000)
cv2.destroyAllWindows()

# 油画效果

- 步骤：
    1. 转化成灰度
    2. 将图片分割成若干个小方块(7x7,10x10)，统计小方块中，每个像素的灰度值
    3. 将0-255分几个灰度段，并将第2步的灰度值，映射到每个灰度段中
    4. 找到数量最多的灰度段，求出均值。完成灰度段中，像素的个数统计
    5. 用统计出来的平均值，代替原来的像素值，实现优化效果

In [47]:
# 运算量大


# 定义8个灰度段，每段等级差为32
# 256/8 = 32; 0-31, 32-63, 64-95, 96-127……

# 定义8x8的小方块

# 当前的像素值，完成了累加

import time

img = cv2.imread("/Users/zhengtaizhong/Desktop/pic.jpg",1)
height = img.shape[0]
width = img.shape[1]
gray_cvt = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

dst_canvas = np.zeros((height,width,3), np.uint8)

time1 = time.time()
for i in range(4,height-4):
    for j in range(4, width-4):
        
        gray_interval = np.zeros(8,np.uint8)    # 定义8个灰度段，0-7
        for m in range(-4,4):
            for n in range(-4,4):
                p1 = int(gray_cvt[i+m, j+n]/32)    # 当前像素属于哪一个灰度段
                gray_interval[p1] = gray_interval[p1] + 1     # 在8个灰度段(0-7)中，【统计】像素数量最多的灰度段
        
        # 在8个灰度段中，【获取】像素数最多的灰度段
        max_amount = gray_interval[0]
        l = 0    # 灰度段下标
        for k in range(0,8):
            if max_amount < gray_interval[k]:
                max_amount = gray_interval[k]
                l = k    # 当前的像素处于的灰度段
        
        # 简化：取出每块的最大值。正常：平均值
        for m in range(-4,4):
            for n in range(-4,4):
                if gray_cvt[i+m, j+n] >= (l*32) and gray_cvt[i+m, j+n] <= ((l+1)*32):
                    (b,g,r) = img[i+m,j+m]
        
        dst_canvas[i,j] = (b,g,r)
        
time2 = time.time()
print("耗时：", time2-time1)

cv2.imshow("orignal", img)
cv2.imshow("canvas", dst_canvas)
cv2.waitKey(5000)
cv2.destroyAllWindows()

耗时： 393.08250093460083
